### Prostate dataset

### Overview

Only downscale, no other operations




In [1]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk
import sys
sys.path.insert(0, '../../dataloaders/')
import niftiio as nio

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [7]:
DOMAINS = os.listdir('./tmp_reindexed/')
DM = DOMAINS[0]
IMG_FOLDER = f'./tmp_reindexed/{DM}/'
SEG_FOLDER = IMG_FOLDER
imgs = glob.glob(IMG_FOLDER + "/image_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/label_*.nii.gz")) ]

pids = [pid.split("_")[-1].split(".")[0] for pid in imgs]

In [8]:
imgs

['./tmp_reindexed/D/image_1.nii.gz',
 './tmp_reindexed/D/image_26.nii.gz',
 './tmp_reindexed/D/image_27.nii.gz',
 './tmp_reindexed/D/image_28.nii.gz',
 './tmp_reindexed/D/image_29.nii.gz',
 './tmp_reindexed/D/image_30.nii.gz',
 './tmp_reindexed/D/image_31.nii.gz',
 './tmp_reindexed/D/image_32.nii.gz',
 './tmp_reindexed/D/image_33.nii.gz',
 './tmp_reindexed/D/image_34.nii.gz',
 './tmp_reindexed/D/image_35.nii.gz',
 './tmp_reindexed/D/image_36.nii.gz',
 './tmp_reindexed/D/image_37.nii.gz']

In [9]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    #resample.SetSize( [int(sz+1) for sz in new_size] )
    resample.SetSize( [int(sz) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj
        
## Then crop ROI
def get_label_center(label):
    nnz = np.sum(label > 1e-5)
    return np.int32(np.rint(np.sum(np.nonzero(label), axis = 1) * 1.0 / nnz))

def image_crop(ori_vol, crop_size, referece_ctr_idx, padval = 0., only_2d = True):
    """ crop a 3d matrix given the index of the new volume on the original volume
        Args:
            refernce_ctr_idx: the center of the new volume on the original volume (in indices)
            only_2d: only do cropping on first two dimensions
    """
    _expand_cropsize = [x + 1 for x in crop_size] # to deal with boundary case
    if only_2d:
        assert len(crop_size) == 2, "Actual len {}".format(len(crop_size))
        assert len(referece_ctr_idx) == 2, "Actual len {}".format(len(referece_ctr_idx))
        _expand_cropsize.append(ori_vol.shape[-1])
        
    image_patch = np.ones(tuple(_expand_cropsize)) * padval

    half_size = tuple( [int(x * 1.0 / 2) for x in _expand_cropsize] )
    _min_idx = [0,0,0]
    _max_idx = list(ori_vol.shape)

    # bias of actual cropped size to the beginning and the end of this volume
    _bias_start = [0,0,0]
    _bias_end = [0,0,0]

    for dim,hsize in enumerate(half_size):
        if dim == 2 and only_2d:
            break

        _bias_start[dim] = np.min([hsize, referece_ctr_idx[dim]])
        _bias_end[dim] = np.min([hsize, ori_vol.shape[dim] - referece_ctr_idx[dim]])

        _min_idx[dim] = referece_ctr_idx[dim] - _bias_start[dim]
        _max_idx[dim] = referece_ctr_idx[dim] + _bias_end[dim]
        
    if only_2d:
        image_patch[ half_size[0] - _bias_start[0]: half_size[0] +_bias_end[0], \
                half_size[1] - _bias_start[1]: half_size[1] +_bias_end[1], ... ] = \
                ori_vol[ referece_ctr_idx[0] - _bias_start[0]: referece_ctr_idx[0] +_bias_end[0], \
                referece_ctr_idx[1] - _bias_start[1]: referece_ctr_idx[1] +_bias_end[1], ... ]

        image_patch = image_patch[ 0: crop_size[0], 0: crop_size[1], : ]
    # then goes back to original volume
    else:
        image_patch[ half_size[0] - _bias_start[0]: half_size[0] +_bias_end[0], \
                half_size[1] - _bias_start[1]: half_size[1] +_bias_end[1], \
                half_size[2] - _bias_start[2]: half_size[2] +_bias_end[2] ] = \
                ori_vol[ referece_ctr_idx[0] - _bias_start[0]: referece_ctr_idx[0] +_bias_end[0], \
                referece_ctr_idx[1] - _bias_start[1]: referece_ctr_idx[1] +_bias_end[1], \
                referece_ctr_idx[2] - _bias_start[2]: referece_ctr_idx[2] +_bias_end[2] ]

        image_patch = image_patch[ 0: crop_size[0], 0: crop_size[1], 0: crop_size[2] ]
    return image_patch

   
    
def copy_spacing_ori(src, dst):
    dst.SetSpacing(src.GetSpacing())
    dst.SetOrigin(src.GetOrigin())
    dst.SetDirection(src.GetDirection())
    return dst

In [10]:
imgs

['./tmp_reindexed/D/image_1.nii.gz',
 './tmp_reindexed/D/image_26.nii.gz',
 './tmp_reindexed/D/image_27.nii.gz',
 './tmp_reindexed/D/image_28.nii.gz',
 './tmp_reindexed/D/image_29.nii.gz',
 './tmp_reindexed/D/image_30.nii.gz',
 './tmp_reindexed/D/image_31.nii.gz',
 './tmp_reindexed/D/image_32.nii.gz',
 './tmp_reindexed/D/image_33.nii.gz',
 './tmp_reindexed/D/image_34.nii.gz',
 './tmp_reindexed/D/image_35.nii.gz',
 './tmp_reindexed/D/image_36.nii.gz',
 './tmp_reindexed/D/image_37.nii.gz']

In [11]:
segs

['./tmp_reindexed/D/label_1.nii.gz',
 './tmp_reindexed/D/label_26.nii.gz',
 './tmp_reindexed/D/label_27.nii.gz',
 './tmp_reindexed/D/label_28.nii.gz',
 './tmp_reindexed/D/label_29.nii.gz',
 './tmp_reindexed/D/label_30.nii.gz',
 './tmp_reindexed/D/label_31.nii.gz',
 './tmp_reindexed/D/label_32.nii.gz',
 './tmp_reindexed/D/label_33.nii.gz',
 './tmp_reindexed/D/label_34.nii.gz',
 './tmp_reindexed/D/label_35.nii.gz',
 './tmp_reindexed/D/label_36.nii.gz',
 './tmp_reindexed/D/label_37.nii.gz']

In [12]:
DOMAINS

['D', 'C', 'A', 'F', 'B', 'E']

In [13]:
import copy
OUT_FOLDER = "./processed/"
HIST_CUT_TOP = 0.5


for DM in DOMAINS:
    scan_dir = os.path.join(OUT_FOLDER, DM)
    os.makedirs(scan_dir, exist_ok = True)

    SPA_FAC = (384) * 1.0 / 192 # spacing factor
    
    
    IMG_FOLDER = f'./tmp_reindexed/{DM}/'
    SEG_FOLDER = IMG_FOLDER
    imgs = glob.glob(IMG_FOLDER + "/image_*.nii.gz")
    imgs = [ fid for fid in sorted(imgs) ]
    segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/label_*.nii.gz")) ]

    pids = [pid.split("_")[-1].split(".")[0] for pid in imgs]
    

    for img_fid, seg_fid, pid in zip(imgs, segs, pids):

        
        lb_n = nio.read_nii_bysitk(seg_fid)

        img_obj = sitk.ReadImage( img_fid )
        seg_obj = sitk.ReadImage( seg_fid )

        ## image
        array = sitk.GetArrayFromImage(img_obj)
        
        _tmp_seg = sitk.GetArrayFromImage(seg_obj)
        
        
        
        # histogram cut
        hir = float(np.percentile(array, 100.0 - HIST_CUT_TOP))
        array[array > hir] = hir
        
        cropped_img_o = sitk.GetImageFromArray(array)
        cropped_img_o = copy_spacing_ori(img_obj, cropped_img_o)

        # resampling
        img_spa_ori = img_obj.GetSpacing()
        res_img_o = resample_by_res(cropped_img_o, [img_spa_ori[0] * SPA_FAC, img_spa_ori[1] * SPA_FAC, img_spa_ori[-1]], interpolator = sitk.sitkLinear,
                                        logging = True)

        ## label
        lb_arr = sitk.GetArrayFromImage(seg_obj)

        
        # label merging, following Liu et al., MICCAI'20
        lb_arr[lb_arr > 1] = 1
        cropped_lb_o = sitk.GetImageFromArray(lb_arr)
        cropped_lb_o = copy_spacing_ori(seg_obj, cropped_lb_o)

        lb_spa_ori = seg_obj.GetSpacing()

        # resampling
        res_lb_o = resample_lb_by_res(cropped_lb_o,  [lb_spa_ori[0] * SPA_FAC, lb_spa_ori[1] * SPA_FAC, lb_spa_ori[-1] ], interpolator = sitk.sitkLinear,
                                      ref_img = res_img_o, logging = True)


        out_img_fid = os.path.join( scan_dir, f'image_{pid}.nii.gz' )
        out_lb_fid  = os.path.join( scan_dir, f'label_{pid}.nii.gz' ) 

        # then save
        sitk.WriteImage(res_img_o, out_img_fid, True) 
        sitk.WriteImage(res_lb_o, out_lb_fid, True) 
        print("{} has been saved".format(out_img_fid))
        print("{} has been saved".format(out_lb_fid))


Spacing: (0.46875065565109253, 0.4687559902667999, 3.0002601146698) -> [0.9375013113021851, 0.9375119805335999, 3.0002601146698]
Size (384, 384, 29) -> [192. 192.  29.]
Label values: [0. 1.]
Spacing: (0.46875065565109253, 0.4687559902667999, 3.0002601146698) -> [0.9375013113021851, 0.9375119805335999, 3.0002601146698]
Size (384, 384, 29) -> [192. 192.  29.]
Spacing: (0.46875065565109253, 0.4687559902667999, 3.0002601146698) -> [0.9375013113021851, 0.9375119805335999, 3.0002601146698]
Size (384, 384, 29) -> [192. 192.  29.]
./processed/D/image_1.nii.gz has been saved
./processed/D/label_1.nii.gz has been saved
Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [1.0416666269302368, 1.0416666269302368, 3.299999952316284]
Size (384, 384, 23) -> [192. 192.  23.]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.299999952316284) -> [1.0416666269302368, 1.0416666269302368, 3.299999952316284]
Size (384, 384, 23) -> [192. 192.  23.]
Spacing: (0.52083

./processed/C/image_0.nii.gz has been saved
./processed/C/label_0.nii.gz has been saved
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2499675750732422) -> [1.0685220956802368, 0.8570437431335449, 1.2499675750732422]
Size (384, 384, 64) -> [192. 192.  64.]
Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2499675750732422) -> [1.0685220956802368, 0.8570437431335449, 1.2499675750732422]
Size (384, 384, 64) -> [192. 192.  64.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2499675750732422) -> [1.0685220956802368, 0.8570437431335449, 1.2499675750732422]
Size (384, 384, 64) -> [192. 192.  64.]
./processed/C/image_1.nii.gz has been saved
./processed/C/label_1.nii.gz has been saved
Spacing: (0.5342610478401184, 0.42852187156677246, 1.250106692314148) -> [1.0685220956802368, 0.8570437431335449, 1.250106692314148]
Size (384, 384, 64) -> [192. 192.  64.]
Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.250106692314148) -> [1.068522

./processed/C/image_4.nii.gz has been saved
./processed/C/label_4.nii.gz has been saved
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500154972076416) -> [1.0685220956802368, 0.8570437431335449, 1.2500154972076416]
Size (384, 384, 64) -> [192. 192.  64.]
Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500154972076416) -> [1.0685220956802368, 0.8570437431335449, 1.2500154972076416]
Size (384, 384, 64) -> [192. 192.  64.]
Spacing: (0.5342610478401184, 0.42852187156677246, 1.2500154972076416) -> [1.0685220956802368, 0.8570437431335449, 1.2500154972076416]
Size (384, 384, 64) -> [192. 192.  64.]
./processed/C/image_5.nii.gz has been saved
./processed/C/label_5.nii.gz has been saved
Spacing: (0.5342610478401184, 0.4388573169708252, 1.2499207258224487) -> [1.0685220956802368, 0.8777146339416504, 1.2499207258224487]
Size (384, 384, 64) -> [192. 192.  64.]
Label values: [0. 1.]
Spacing: (0.5342610478401184, 0.4388573169708252, 1.2499207258224487) -> [1.06852

./processed/A/image_16.nii.gz has been saved
./processed/A/label_16.nii.gz has been saved
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999984741210938) -> [1.0416666269302368, 1.0416666269302368, 3.5999984741210938]
Size (384, 384, 20) -> [192. 192.  20.]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999984741210938) -> [1.0416666269302368, 1.0416666269302368, 3.5999984741210938]
Size (384, 384, 20) -> [192. 192.  20.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999984741210938) -> [1.0416666269302368, 1.0416666269302368, 3.5999984741210938]
Size (384, 384, 20) -> [192. 192.  20.]
./processed/A/image_17.nii.gz has been saved
./processed/A/label_17.nii.gz has been saved
Spacing: (0.5, 0.5, 4.000001907348633) -> [1.0, 1.0, 4.000001907348633]
Size (384, 384, 18) -> [192. 192.  18.]
Label values: [0. 1.]
Spacing: (0.5, 0.5, 4.000000953674316) -> [1.0, 1.0, 4.000000953674316]
Size (384, 384, 18) -> [192. 192.  18.]
Spacing: (0.5, 0.5, 4.00000095

./processed/A/image_29.nii.gz has been saved
./processed/A/label_29.nii.gz has been saved
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000022888183594) -> [1.0416666269302368, 1.0416666269302368, 3.6000022888183594]
Size (384, 384, 20) -> [192. 192.  20.]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000022888183594) -> [1.0416666269302368, 1.0416666269302368, 3.6000022888183594]
Size (384, 384, 20) -> [192. 192.  20.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000022888183594) -> [1.0416666269302368, 1.0416666269302368, 3.6000022888183594]
Size (384, 384, 20) -> [192. 192.  20.]
./processed/A/image_3.nii.gz has been saved
./processed/A/label_3.nii.gz has been saved
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000008583068848) -> [1.0416666269302368, 1.0416666269302368, 3.6000008583068848]
Size (384, 384, 20) -> [192. 192.  20.]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.6000006198883057) -> [1.041666

./processed/F/image_44.nii.gz has been saved
./processed/F/label_44.nii.gz has been saved
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [1.0416666269302368, 1.0416666269302368, 3.5999999046325684]
Size (384, 384, 24) -> [192. 192.  24.]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [1.0416666269302368, 1.0416666269302368, 3.5999999046325684]
Size (384, 384, 24) -> [192. 192.  24.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [1.0416666269302368, 1.0416666269302368, 3.5999999046325684]
Size (384, 384, 24) -> [192. 192.  24.]
./processed/F/image_45.nii.gz has been saved
./processed/F/label_45.nii.gz has been saved
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [1.0416666269302368, 1.0416666269302368, 3.5999999046325684]
Size (384, 384, 24) -> [192. 192.  24.]
Label values: [0. 1.]
Spacing: (0.5208333134651184, 0.5208333134651184, 3.5999999046325684) -> [1.0416

Spacing: (0.4166666865348816, 0.4166666865348816, 3.000000238418579) -> [0.8333333730697632, 0.8333333730697632, 3.000000238418579]
Size (384, 384, 32) -> [192. 192.  32.]
Label values: [0. 1.]
Spacing: (0.4166666865348816, 0.4166666567325592, 3.000000238418579) -> [0.8333333730697632, 0.8333333134651184, 3.000000238418579]
Size (384, 384, 32) -> [192. 192.  32.]
Spacing: (0.4166666865348816, 0.4166666567325592, 3.000000238418579) -> [0.8333333730697632, 0.8333333134651184, 3.000000238418579]
Size (384, 384, 32) -> [192. 192.  32.]
./processed/B/image_16.nii.gz has been saved
./processed/B/label_16.nii.gz has been saved
Spacing: (0.4166666865348816, 0.4166666865348816, 4.000000476837158) -> [0.8333333730697632, 0.8333333730697632, 4.000000476837158]
Size (384, 384, 23) -> [192. 192.  23.]
Label values: [0. 1.]
Spacing: (0.4166666567325592, 0.4166666567325592, 4.000000476837158) -> [0.8333333134651184, 0.8333333134651184, 4.000000476837158]
Size (384, 384, 23) -> [192. 192.  23.]
Spacin

./processed/B/image_28.nii.gz has been saved
./processed/B/label_28.nii.gz has been saved
Spacing: (0.4427083134651184, 0.4427083134651184, 2.9999983310699463) -> [0.8854166269302368, 0.8854166269302368, 2.9999983310699463]
Size (384, 384, 32) -> [192. 192.  32.]
Label values: [0. 1.]
Spacing: (0.4427083432674408, 0.4427083134651184, 2.9999983310699463) -> [0.8854166865348816, 0.8854166269302368, 2.9999983310699463]
Size (384, 384, 32) -> [192. 192.  32.]
Spacing: (0.4427083432674408, 0.4427083134651184, 2.9999983310699463) -> [0.8854166865348816, 0.8854166269302368, 2.9999983310699463]
Size (384, 384, 32) -> [192. 192.  32.]
./processed/B/image_29.nii.gz has been saved
./processed/B/label_29.nii.gz has been saved
Spacing: (0.4166666865348816, 0.4166666865348816, 3.000002145767212) -> [0.8333333730697632, 0.8333333730697632, 3.000002145767212]
Size (384, 384, 32) -> [192. 192.  32.]
Label values: [0. 1.]
Spacing: (0.4166666865348816, 0.4166666567325592, 3.000002145767212) -> [0.8333333

Spacing: (0.36458534002304077, 0.3645839989185333, 2.2000200748443604) -> [0.7291706800460815, 0.7291679978370667, 2.2000200748443604]
Size (384, 384, 42) -> [192. 192.  42.]
Label values: [0. 1.]
Spacing: (0.36458534002304077, 0.3645839989185333, 2.2000200748443604) -> [0.7291706800460815, 0.7291679978370667, 2.2000200748443604]
Size (384, 384, 42) -> [192. 192.  42.]
Spacing: (0.36458534002304077, 0.3645839989185333, 2.2000200748443604) -> [0.7291706800460815, 0.7291679978370667, 2.2000200748443604]
Size (384, 384, 42) -> [192. 192.  42.]
./processed/E/image_3.nii.gz has been saved
./processed/E/label_3.nii.gz has been saved
Spacing: (0.3645866811275482, 0.36458534002304077, 2.200500011444092) -> [0.7291733622550964, 0.7291706800460815, 2.200500011444092]
Size (384, 384, 46) -> [192. 192.  46.]
Label values: [0. 1.]
Spacing: (0.3645866811275482, 0.36458534002304077, 2.200500011444092) -> [0.7291733622550964, 0.7291706800460815, 2.200500011444092]
Size (384, 384, 46) -> [192. 192.  46

### sanity check passed